In [18]:
import numpy as np
import pandas as pd
from pyomo.environ import *
import time

np.random.seed(30)

# Start the timer to track runtime
start_time = time.time()

# Load distance matrices from Excel
file_path = "distance_matrice.xlsx"
transmission_distances_df = pd.read_excel(file_path, sheet_name="Transmission_Costs", index_col=0)
travel_distances_df = pd.read_excel(file_path, sheet_name="Travel_Costs", index_col=0)

# Define per-mile costs
transmission_cost_per_mile = 0.04  # $/kWh per mile
travel_cost_per_mile = 0.70        # $/mile

# Define sets
I = list(travel_distances_df.index)  # Charging station locations
J = list(travel_distances_df.columns)  # Demand locations
K = list(transmission_distances_df.index)  # Power substations

# Convert distance matrices to cost matrices
T = transmission_distances_df.to_numpy() * transmission_cost_per_mile  # Transmission costs
C = travel_distances_df.to_numpy() * travel_cost_per_mile  # Travel costs

# Parameters
d_j = {j: np.random.randint(20, 29) for j in J}  # Random demand values
C_i = {i: 100 for i in I}  # Charging station capacity =100 kWh to 350 kWh
P_k = {k: 300 for k in K}  # Substation capacity=300 = 20,000 kWh
S_k = {k: 50 for k in K}  # Existing substation demand

# Binary coverage parameters based on threshold
a_ij = {(i, j): 1 if C[i_idx, j_idx] <= 100 else 0 for i_idx, i in enumerate(I) for j_idx, j in enumerate(J)}
b_ik = {(i, k): 1 if T[k_idx, i_idx] <= 100 else 0 for k_idx, k in enumerate(K) for i_idx, i in enumerate(I)}

# Create the Pyomo model
model = ConcreteModel()

# Decision Variables
model.x = Var(I, domain=Binary)  # Charging station decision
model.y = Var(J, domain=Binary)  # Demand coverage decision
model.z = Var([(i, k) for i in I for k in K], domain=Binary)  # Substation-station connection decision

# Objective Function: Maximize demand coverage
model.obj = Objective(expr=sum(d_j[j] * model.y[j] for j in J), sense=maximize)

# Constraints

# 1. Budget constraint
Q = 80000  # Total budget
F = 2000  # Fixed cost per charging station
model.budget = Constraint(expr=sum(F * model.x[i] for i in I) <= Q)

# 2. Coverage constraint
def coverage_rule(model, j):
    return model.y[j] <= sum(a_ij[i, j] * model.x[i] for i in I)
model.coverage = Constraint(J, rule=coverage_rule)

# 3. Charging station capacity constraint
def station_capacity_rule(model, i):
    return sum(a_ij[i, j] * d_j[j] * model.y[j] for j in J) <= C_i[i] * model.x[i]
model.station_capacity = Constraint(I, rule=station_capacity_rule)

# 4. Substation capacity constraint
def substation_capacity_rule(model, k):
    return sum(model.z[i, k] * C_i[i] for i in I) + S_k[k] <= P_k[k]
model.substation_capacity = Constraint(K, rule=substation_capacity_rule)

# 5. Substation feasibility constraint
def feasibility_rule(model, i, k):
    return model.z[i, k] <= b_ik[i, k]
model.feasibility = Constraint([(i, k) for i in I for k in K], rule=feasibility_rule)

# Solve the model
solver = SolverFactory('glpk')  # Using GLPK solver
solver.solve(model)

# End the timer to track runtime
end_time = time.time()
runtime = end_time - start_time

# Count the number of open stations
num_open_stations = sum(1 for i in I if model.x[i]() > 0.5)

# Output results
print("Objective value (Total demand covered):", model.obj())
print("\nCharging station placement:")
for i in I:
    print(f"  Station {i}: {'Open' if model.x[i]() > 0.5 else 'Closed'}")

print("\nDemand coverage:")
for j in J:
    print(f"  Demand {j}: {'Covered' if model.y[j]() > 0 else 'Not Covered'}")

print("\nSubstation connections:")
for i in I:
    for k in K:
        if model.z[i, k]() > 0:
            print(f"  Station {i} is connected to Substation {k}")

print(f"\nNumber of open stations: {num_open_stations}")
print(f"Model runtime: {runtime:.2f} seconds")


Objective value (Total demand covered): 22.0

Charging station placement:
  Station Demand_1: Open
  Station Demand_2: Open
  Station Demand_3: Open
  Station Demand_4: Open
  Station Demand_5: Open
  Station Demand_6: Open
  Station Demand_7: Open
  Station Demand_8: Open
  Station Demand_9: Open
  Station Demand_10: Open
  Station Demand_11: Open
  Station Demand_12: Open
  Station Demand_13: Open
  Station Demand_14: Open
  Station Demand_15: Open
  Station Demand_16: Open
  Station Demand_17: Open
  Station Demand_18: Open
  Station Demand_19: Open
  Station Demand_20: Open
  Station Demand_21: Open
  Station Demand_22: Open
  Station Demand_23: Open
  Station Demand_24: Open
  Station Demand_25: Open
  Station Demand_26: Open
  Station Demand_27: Open
  Station Demand_28: Open
  Station Demand_29: Open
  Station Demand_30: Open
  Station Demand_31: Open
  Station Demand_32: Open
  Station Demand_33: Closed
  Station Demand_34: Closed
  Station Demand_35: Closed
  Station Demand_36